# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [131]:
# your code here
orders = pd.read_csv("../data/Orders.csv")
orders.head()


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [132]:
orders["CustomerID"].unique()

array([17850, 13047, 12583, ..., 13298, 14569, 12713])

In [168]:
orders.shape

(397924, 14)

In [133]:
orders["CustomerID"].value_counts()

17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
         ... 
17331       1
16093       1
13703       1
13307       1
12346       1
Name: CustomerID, Length: 4339, dtype: int64

In [134]:
cust_spent = orders.groupby("CustomerID")["amount_spent"].sum()

In [135]:
type(cust_spent)

pandas.core.series.Series

In [136]:
orders2 = orders.merge(right = cust_spent, how = "left", on = "CustomerID")

In [137]:
orders2.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21


In [138]:
orders2["Quartiles"] = pd.qcut(orders2["amount_spent_y"], q=[0.,0.75,0.95,1.], labels=["Regular","Preferred","VIP"])

In [139]:
orders2.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Quartiles
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Regular
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Regular
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Regular
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Regular
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Regular


In [140]:
orders2.sort_values("amount_spent_y",ascending=False)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Quartiles
319037,434742,574059,22725,2011,11,3,14,alarm clock bakelike chocolate,1,2011-11-02 14:13:00,3.75,14646,Netherlands,3.75,280206.02,VIP
319000,434703,574056,22384,2011,11,3,14,lunch bag pink polkadot,100,2011-11-02 14:09:00,1.45,14646,Netherlands,145.00,280206.02,VIP
319018,434723,574059,20979,2011,11,3,14,36 pencils tube red retrospot,1,2011-11-02 14:13:00,1.25,14646,Netherlands,1.25,280206.02,VIP
319017,434722,574059,23507,2011,11,3,14,mini playing cards buffalo bill,1,2011-11-02 14:13:00,0.42,14646,Netherlands,0.42,280206.02,VIP
319016,434721,574059,23510,2011,11,3,14,mini playing cards gymkhana,1,2011-11-02 14:13:00,0.42,14646,Netherlands,0.42,280206.02,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306187,419485,572832,M,2011,10,3,11,manual,1,2011-10-26 11:40:00,1.00,16454,United Kingdom,1.00,6.90,Regular
276665,383198,570003,22371,2011,10,5,9,airline bag vintage tokyo 78,1,2011-10-07 09:19:00,4.25,14792,United Kingdom,4.25,6.20,Regular
276664,383197,570003,47594A,2011,10,5,9,carousel design washbag,1,2011-10-07 09:19:00,1.95,14792,United Kingdom,1.95,6.20,Regular
56063,90155,543986,46000U,2011,2,2,9,polyester filler pad 30cmx30cm,3,2011-02-15 09:46:00,1.25,16738,United Kingdom,3.75,3.75,Regular


In [167]:
orders2.groupby("Quartiles")["Quartiles"].value_counts()

Quartiles  Quartiles
Regular    Regular      298527
Preferred  Preferred     81591
VIP        VIP           17806
Name: Quartiles, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [150]:
VIP = orders2["Quartiles"]=="VIP"

In [152]:
orders_VIP = orders2[VIP]

In [160]:
orders_VIP.groupby("Country")["Country"].value_counts().sort_values(ascending=False)

Country         Country       
United Kingdom  United Kingdom    7933
EIRE            EIRE              7077
Netherlands     Netherlands       2080
Australia       Australia          716
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [161]:
Preferred_VIP = (orders2["Quartiles"]=="VIP") | (orders2["Quartiles"]=="Preferred")

In [162]:
orders_VIP_Preferred = orders2[Preferred_VIP]

In [163]:
orders_VIP_Preferred.groupby("Country")["Country"].value_counts().sort_values(ascending=False)

Country          Country        
United Kingdom   United Kingdom     79619
EIRE             EIRE                7077
Germany          Germany             3127
France           France              2900
Netherlands      Netherlands         2080
Australia        Australia            898
Portugal         Portugal             681
Spain            Spain                511
Switzerland      Switzerland          463
Norway           Norway               420
Channel Islands  Channel Islands      364
Finland          Finland              294
Cyprus           Cyprus               248
Singapore        Singapore            222
Japan            Japan                205
Sweden           Sweden               198
Belgium          Belgium               54
Denmark          Denmark               36
Name: Country, dtype: int64